In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark import SparkContext
import matplotlib.pyplot as plt
import numpy as np

In [2]:
spark = SparkSession \
    .builder \
    .appName("Spark PJ") \
    .getOrCreate()
spark.conf.set("spark.executor.memory", "32g")
spark.conf.set('spark.driver.memory','15g')

In [3]:
filePath = "./mernis/data_dump.sql"
# the method below is going to make your memory explode!!!
"""data=[]
with open(filePath, 'r') as f:    
    for i in range(75):
        f.readline()
    for i in range(12000000):
        data.append(f.readline().strip().split('\t'))"""

"data=[]\nwith open(filePath, 'r') as f:    \n    for i in range(75):\n        f.readline()\n    for i in range(12000000):\n        data.append(f.readline().strip().split('\t'))"

In [4]:
sc = SparkContext.getOrCreate()
data = sc.textFile(filePath,4).map(lambda l:l.split('\t'))
data = data.filter(lambda l: len(l)>1)

In [5]:
schema=['uid', 'national_identifier', 'first_name', "last_name",'mother_first','father_first', 'gender', 'birth_city',
'date_of_birth', 'id_registration_city', 'id_registration_district', 'address_city', 'address_district',
'address_neighborhood', 'street_address', 'door_or_entrance_number', 'misc']
df = spark.createDataFrame(data, schema)

In [6]:
df.createOrReplaceTempView("table")

In [32]:
df.head(5)

[Row(uid=u'291990', national_identifier=u'23480340824', first_name=u'NESLIHAN', last_name=u'ZENGIN', mother_first=u'ZEYCAN', father_first=u'OSMAN', gender=u'K', birth_city=u'KANGAL', date_of_birth=u'10/6/1978', id_registration_city=u'MALATYA', id_registration_district=u'KULUNCAK', address_city=u'MALATYA', address_district=u'KULUNCAK', address_neighborhood=u'ISMETPASA MAH.', street_address=u'BOGAZICI CADDESI', door_or_entrance_number=u'14', misc=u'<NULL>'),
 Row(uid=u'291991', national_identifier=u'17111553172', first_name=u'SADET', last_name=u'YILDIRIM', mother_first=u'ZOHRE', father_first=u'ISMAIL', gender=u'K', birth_city=u'MERSIN', date_of_birth=u'3/8/1949', id_registration_city=u'MALATYA', id_registration_district=u'KULUNCAK', address_city=u'MALATYA', address_district=u'KULUNCAK', address_neighborhood=u'ISMETPASA MAH.', street_address=u'CITILBAGI SOKAK', door_or_entrance_number=u'40', misc=u'<NULL>'),
 Row(uid=u'291992', national_identifier=u'10499773538', first_name=u'GONUL', last

## E1:统计土耳其公民中所有人中年龄最大的男人

In [29]:
maxAge = spark.sql("SELECT uid,2019 - INT(SUBSTRING(date_of_birth,-4,4)) AS age FROM table ORDER BY age desc limit 5")

maxAge.show()

+--------+---+
|     uid|age|
+--------+---+
|30759557|701|
|27821738|696|
|29540857|693|
|32198722|690|
|24516674|689|
+--------+---+



In [46]:
type(maxAge)

pyspark.sql.dataframe.DataFrame

## E2. 统计姓名中最常出现的字母

In [51]:
# name = spark.sql("SELECT first_name as fn, last_name as ln  FROM table")
def spl(x):
    y=x.split()
    z=list()
    for i in y:
        z.extend(list(i))
    return z
def spl2(x):
    return x.split()
name = spark.sql("SELECT concat(first_name , last_name) FROM table")
nameLetters=name.rdd.flatMap(lambda x:x).flatMap(lambda x:spl(x))

sortByLetter=nameLetters.map(lambda x: (x, 1)). reduceByKey(lambda x,y: x+y). sortBy(lambda x: x[1], False)

sortByLetter.take(30)

[(u'A', 82319942),
 (u'E', 56600565),
 (u'I', 49279256),
 (u'N', 37773093),
 (u'R', 35681543),
 (u'U', 34389962),
 (u'L', 32581810),
 (u'M', 29708399),
 (u'S', 29110326),
 (u'K', 27433586),
 (u'T', 24482238),
 (u'Y', 21262594),
 (u'C', 16788753),
 (u'H', 15947044),
 (u'O', 15731863),
 (u'D', 15259507),
 (u'G', 14257687),
 (u'Z', 14141151),
 (u'B', 10725429),
 (u'F', 7402885),
 (u'V', 4901782),
 (u'P', 3693601),
 (u'J', 134385),
 (u'.', 12177),
 (u'W', 1390),
 (u'-', 709),
 (u'X', 423),
 (u'Q', 205),
 (u'(', 175),
 (u')', 174)]

## E3: 统计该国人又的年龄分布,年龄段分(0-18, 19-28, 29-38,39-48, 49-59,>60)

In [54]:
spark.sql("select COUNT(*) from (SELECT uid,2019 - INT(SUBSTRING(date_of_birth,-4,4)) AS age FROM table) as NEW WHERE NEW.age>=0 AND NEW.age<=18").show()
# spark.sql("SELECT uid,2019 - INT(SUBSTRING(date_of_birth,-4,4)) AS age, COUNT(*) FROM table WHERE age>=19 AND age<=28").show()
# spark.sql("SELECT uid,2019 - INT(SUBSTRING(date_of_birth,-4,4)) AS age, COUNT(*) FROM table WHERE age>=29 AND age<=38").show()
# spark.sql("SELECT uid,2019 - INT(SUBSTRING(date_of_birth,-4,4)) AS age, COUNT(*) FROM table WHERE age>=39 AND age<=48").show()
# spark.sql("SELECT uid,2019 - INT(SUBSTRING(date_of_birth,-4,4)) AS age, COUNT(*) FROM table WHERE age>=49 AND age<=59").show()
# spark.sql("SELECT uid,2019 - INT(SUBSTRING(date_of_birth,-4,4)) AS age, COUNT(*) FROM table WHERE age>=60").show()

+--------+
|count(1)|
+--------+
|       0|
+--------+



## ANOTHER WAY

In [55]:
tmp = spark.sql('SELECT uid,2019 - INT(SUBSTRING(date_of_birth,-4,4)) AS age FROM table')

In [72]:
tmp.select(tmp.age).filter(tmp.age>=0).filter(tmp.age<=18).groupBy("age").count().show()

+---+-----+
|age|count|
+---+-----+
+---+-----+



In [ ]:
tmp.select(tmp.age).filter(tmp.age>=0).filter(tmp.age<=18).count().show()

In [70]:
tmp.select(tmp.age).filter(tmp.age>=19).filter(tmp.age<=28).groupBy("age").count().show()

+---+------+
|age| count|
+---+------+
| 28|349145|
+---+------+



In [108]:
tmp.select(tmp.age).filter(tmp.age>=19).filter(tmp.age<=28).count()

349145

In [71]:
tmp.select(tmp.age).filter(tmp.age>=29).filter(tmp.age<=38).groupBy("age").count().show()

+---+-------+
|age|  count|
+---+-------+
| 31|1045151|
| 34|1258091|
| 37|1298211|
| 35|1272332|
| 38|1424503|
| 29|1267091|
| 32|1242773|
| 33|1262629|
| 30|1271867|
| 36|1250523|
+---+-------+



In [113]:
tmp.select(tmp.age).filter(tmp.age>=29).filter(tmp.age<=38).count()

12593171

In [73]:
tmp.select(tmp.age).filter(tmp.age>=39).filter(tmp.age<=48).groupBy("age").count().show()

+---+-------+
|age|  count|
+---+-------+
| 44|1097350|
| 47|1112227|
| 40|1284212|
| 48|1054244|
| 41|1256054|
| 43|1111588|
| 39|1362790|
| 45|1228699|
| 42|1208954|
| 46|1153070|
+---+-------+



In [109]:
tmp.select(tmp.age).filter(tmp.age>=39).filter(tmp.age<=48).count()

11869188

In [74]:
tmp.select(tmp.age).filter(tmp.age>=49).filter(tmp.age<=59).groupBy("age").count().show()

+---+-------+
|age|  count|
+---+-------+
| 53|1027736|
| 52| 794745|
| 57| 885179|
| 54|1080864|
| 55|1011845|
| 59| 965010|
| 49|1045109|
| 51| 917911|
| 50| 976998|
| 56| 894883|
| 58| 765783|
+---+-------+



In [110]:
tmp.select(tmp.age).filter(tmp.age>=49).filter(tmp.age<=59).count()

10366063

In [75]:
tmp.select(tmp.age).filter(tmp.age>=60).groupBy("age").count().show()

+---+------+
|age| count|
+---+------+
| 85|255012|
| 65|699795|
| 78|306263|
|673|     5|
|108|  2488|
|683|     6|
|101| 12725|
|115|   375|
| 81|314358|
| 76|346370|
|667|     1|
|103|  8885|
| 91|149804|
|685|     1|
|686|   155|
|122|    22|
|625|     2|
| 93|123175|
|111|   751|
|665|     1|
+---+------+
only showing top 20 rows



In [111]:
tmp.select(tmp.age).filter(tmp.age>=60).count()

14434140

## E4: 按月份,统计该国人口生日在每个月上的分布

In [76]:
birth=spark.sql("SELECT INT(split(date_of_birth,'/')[1]) as birth_month,COUNT(*) AS number FROM table GROUP BY birth_month ORDER BY birth_month")
birth.show()

+-----------+-------+
|birth_month| number|
+-----------+-------+
|       null|  61388|
|          0|    673|
|          1|7824976|
|          2|4688993|
|          3|5124982|
|          4|4184062|
|          5|4217321|
|          6|3463381|
|          7|4249153|
|          8|3170640|
|          9|3450705|
|         10|3434428|
|         11|2912085|
|         12|2828919|
|         14|      1|
|         15|      2|
+-----------+-------+



## E5: 统计一下该国的男女比例,男女人数

In [77]:
spark.sql("SELECT gender,COUNT(*) AS number FROM table GROUP BY gender").show()

+------+--------+
|gender|  number|
+------+--------+
|     K|25077226|
|     E|24534483|
+------+--------+



## N1: 统计男性,女性最常见的10个姓

## 女

In [78]:
spark.sql("SELECT last_name,COUNT(*) AS count FROM table WHERE gender='E' GROUP BY last_name ORDER BY count DESC").show(10)

+---------+------+
|last_name| count|
+---------+------+
|   YILMAZ|352338|
|     KAYA|244272|
|    DEMIR|231289|
|    SAHIN|201958|
|    CELIK|199622|
|   YILDIZ|195162|
| YILDIRIM|191966|
|   OZTURK|178610|
|    AYDIN|177894|
|  OZDEMIR|164085|
+---------+------+
only showing top 10 rows



## 男

In [79]:
spark.sql("SELECT last_name,COUNT(*) AS count FROM table WHERE gender='K' GROUP BY last_name ORDER BY count DESC").show(10)

+---------+------+
|last_name| count|
+---------+------+
|   YILMAZ|355954|
|     KAYA|244100|
|    DEMIR|230428|
|    SAHIN|202155|
|    CELIK|199330|
|   YILDIZ|194060|
| YILDIRIM|192835|
|   OZTURK|180292|
|    AYDIN|178501|
|  OZDEMIR|165924|
+---------+------+
only showing top 10 rows



## N2:统计每个城市市民的平均年龄

In [80]:
spark.sql("SELECT id_registration_city,AVG(2019 - INT(SUBSTRING(date_of_birth,-4,4))) AS average_age FROM table GROUP BY id_registration_city").show()

+--------------------+------------------+
|id_registration_city|       average_age|
+--------------------+------------------+
|               ADANA|50.885431008504504|
|             DENIZLI|53.521267682141875|
|             TRABZON| 51.71044389796684|
|           BALIKESIR| 54.65844873941054|
|             BILECIK|54.640178202453306|
|             GIRESUN|51.731187731298384|
|              ARTVIN| 52.95359133390531|
|           ZONGULDAK|51.628508505530554|
|            ISTANBUL|55.974292262476624|
|             ERZURUM| 49.56476601418911|
|               IGDIR|48.583227637332406|
|            NEVSEHIR|51.867912184651004|
|              MANISA| 53.77766762080073|
|              BITLIS|  46.6854444129062|
|               BURSA| 53.82169031984749|
|               IZMIR| 55.01943768982517|
|                 VAN| 46.32588101800375|
|             HAKKARI|45.711081303717094|
|                BOLU| 53.81699284228758|
|               CORUM|51.275606763278944|
+--------------------+------------

## N3:说一下该国平均人口最年轻的5个城市

In [81]:
spark.sql('''SELECT id_registration_city,AVG(2019 - INT(SUBSTRING(date_of_birth,-4,4))) AS average_age FROM table GROUP BY id_registration_city ORDER BY average_age ''').show(5)

+--------------------+------------------+
|id_registration_city|       average_age|
+--------------------+------------------+
|             HAKKARI|45.711081303717094|
|              SIRNAK| 45.84987434070204|
|              BATMAN| 46.11198901177727|
|                 VAN| 46.32588101800375|
|                 MUS| 46.48979974733893|
+--------------------+------------------+
only showing top 5 rows



## N4: 统计一下该国前10大人又城市中,每个城市的前3大姓氏

In [90]:
popCity = spark.sql("SELECT id_registration_city,COUNT(*) AS number FROM table GROUP BY id_registration_city ORDER BY number DESC")
popCity.show()

+--------------------+-------+
|id_registration_city| number|
+--------------------+-------+
|            ISTANBUL|1900634|
|               KONYA|1557843|
|               IZMIR|1473066|
|              ANKARA|1287335|
|               BURSA|1251065|
|               SIVAS|1209830|
|              SAMSUN|1195234|
|               AYDIN|1186918|
|               ADANA|1108354|
|           SANLIURFA|1066305|
|              MARDIN|1063690|
|          DIYARBAKIR|1016653|
|             ERZURUM|1010916|
|              MANISA| 992523|
|             TRABZON| 975134|
|               HATAY| 956286|
|                ORDU| 954541|
|           BALIKESIR| 926511|
|             KAYSERI| 889003|
|              MERSIN| 878724|
+--------------------+-------+
only showing top 20 rows



In [99]:
city = popCity.rdd.flatMap(lambda x:x).take(20)
#city=[i for i in d1.take(10)]

In [103]:
for c in city:
    print type(c), c

<type 'unicode'> ISTANBUL
<type 'int'> 1900634
<type 'unicode'> KONYA
<type 'int'> 1557843
<type 'unicode'> IZMIR
<type 'int'> 1473066
<type 'unicode'> ANKARA
<type 'int'> 1287335
<type 'unicode'> BURSA
<type 'int'> 1251065
<type 'unicode'> SIVAS
<type 'int'> 1209830
<type 'unicode'> SAMSUN
<type 'int'> 1195234
<type 'unicode'> AYDIN
<type 'int'> 1186918
<type 'unicode'> ADANA
<type 'int'> 1108354
<type 'unicode'> SANLIURFA
<type 'int'> 1066305


In [104]:
for c in city:
    if type(c)==int: continue
    print(c)
    sql_="SELECT last_name,COUNT(*) AS count FROM table WHERE id_registration_city='"+str(c)+"' GROUP BY last_name ORDER BY COUNT(*) DESC"
    spark.sql(sql_).show(3)

ISTANBUL
+---------+-----+
|last_name|count|
+---------+-----+
|   YILMAZ|15025|
|     KAYA| 8381|
|   OZTURK| 7948|
+---------+-----+
only showing top 3 rows

KONYA
+---------+-----+
|last_name|count|
+---------+-----+
|   YILMAZ| 2716|
|    CELIK| 1861|
|     KAYA| 1843|
+---------+-----+
only showing top 3 rows

IZMIR
+---------+-----+
|last_name|count|
+---------+-----+
|   YILMAZ| 5355|
|     KAYA| 3073|
|   OZTURK| 2732|
+---------+-----+
only showing top 3 rows

ANKARA
+---------+-----+
|last_name|count|
+---------+-----+
|   YILMAZ|10949|
|    SAHIN| 7106|
|   OZTURK| 6506|
+---------+-----+
only showing top 3 rows

BURSA
+---------+-----+
|last_name|count|
+---------+-----+
|   YILMAZ| 4846|
|    AYDIN| 3607|
|   YILDIZ| 2867|
+---------+-----+
only showing top 3 rows

SIVAS
+---------+-----+
|last_name|count|
+---------+-----+
|   YILMAZ| 2985|
| YILDIRIM| 2769|
|   YILDIZ| 2691|
+---------+-----+
only showing top 3 rows

SAMSUN
+---------+-----+
|last_name|count|
+---------+

## N5: 计算一下该国前10大人又城市中,每个城市的人口生日最集中分布的是哪2个月

In [112]:
for c in city:
    if type(c)==int: continue
    print(c)
    sql_="SELECT INT(split(date_of_birth,'/')[1]) as birth_month,COUNT(*) AS count FROM table WHERE id_registration_city='"+str(c)+"' GROUP BY birth_month  ORDER BY COUNT(*) DESC"
    spark.sql(sql_).show(2)

ISTANBUL
+-----------+------+
|birth_month| count|
+-----------+------+
|          7|232190|
|          1|210683|
+-----------+------+
only showing top 2 rows

KONYA
+-----------+------+
|birth_month| count|
+-----------+------+
|          1|233455|
|          3|160557|
+-----------+------+
only showing top 2 rows

IZMIR
+-----------+------+
|birth_month| count|
+-----------+------+
|          1|182326|
|          7|160248|
+-----------+------+
only showing top 2 rows

ANKARA
+-----------+------+
|birth_month| count|
+-----------+------+
|          1|179389|
|          3|135286|
+-----------+------+
only showing top 2 rows

BURSA
+-----------+------+
|birth_month| count|
+-----------+------+
|          1|159285|
|          7|133978|
+-----------+------+
only showing top 2 rows

SIVAS
+-----------+------+
|birth_month| count|
+-----------+------+
|          1|162517|
|          3|129923|
+-----------+------+
only showing top 2 rows

SAMSUN
+-----------+------+
|birth_month| count|
+----

## N6: 统计一下，该国男、女最为常用的5个名字是什么

In [7]:
# 男 E 女 K
print("male")
spark.sql("SELECT first_name,COUNT(*) as count FROM table WHERE gender='E' GROUP BY first_name ORDER BY COUNT(*) DESC").show(5)
print("female")
spark.sql("SELECT first_name,COUNT(*) as count FROM table WHERE gender='K' GROUP BY first_name ORDER BY COUNT(*) DESC").show(5)

male
+----------+-------+
|first_name|  count|
+----------+-------+
|    MEHMET|1172949|
|   MUSTAFA| 898640|
|     AHMET| 719375|
|       ALI| 663121|
|   HUSEYIN| 521223|
+----------+-------+
only showing top 5 rows

female
+----------+-------+
|first_name|  count|
+----------+-------+
|     FATMA|1154707|
|      AYSE| 893025|
|     EMINE| 756629|
|    HATICE| 658979|
|    ZEYNEP| 315488|
+----------+-------+
only showing top 5 rows



## N7: 计算一下该国前10大人又城市中，每个城市的最受欢迎的3个名字是什么

In [9]:
city=[ 'ISTANBUL','KONYA','IZMIR','ANKARA','BURSA','SIVAS','SAMSUN', 'AYDIN', 'ADANA','SANLIURFA']

In [10]:
for c in city:
    print(c)
    sql_l="SELECT first_name,COUNT(*) AS count FROM table WHERE birth_city='"+str(c)+"' GROUP BY first_name ORDER BY COUNT(*) DESC"
    spark.sql(sql_l).show(3)

ISTANBUL
+----------+-----+
|first_name|count|
+----------+-----+
|     MURAT|18693|
|   MUSTAFA|11684|
|    MEHMET|10735|
+----------+-----+
only showing top 3 rows

KONYA
+----------+-----+
|first_name|count|
+----------+-----+
|   MUSTAFA| 9702|
|    MEHMET| 9515|
|      AYSE| 7442|
+----------+-----+
only showing top 3 rows

IZMIR
+----------+-----+
|first_name|count|
+----------+-----+
|     MURAT| 8508|
|   MUSTAFA| 7388|
|    MEHMET| 7280|
+----------+-----+
only showing top 3 rows

ANKARA
+----------+-----+
|first_name|count|
+----------+-----+
|     MURAT|14150|
|   MUSTAFA| 9857|
|    MEHMET| 8960|
+----------+-----+
only showing top 3 rows

BURSA
+----------+-----+
|first_name|count|
+----------+-----+
|   MUSTAFA| 5950|
|    MEHMET| 5882|
|     FATMA| 5810|
+----------+-----+
only showing top 3 rows

SIVAS
+----------+-----+
|first_name|count|
+----------+-----+
|    HATICE| 3209|
|   MUSTAFA| 2949|
|     AHMET| 2901|
+----------+-----+
only showing top 3 rows

SAMSUN
+----

## H1. 构建人所在城市预测模型:根据给定一个人的所有信息(除了所在城市)，预测该人所在的 城市。分析该模型Top1到 Top 5的预测准确度。

## H2. 性别预测模型:根据给定一个人的信息(除了性别)，能否出该人的性别?

In [18]:
from pyspark.ml.feature import StringIndexer
dddf = spark.createDataFrame([

    (0, "a"), (1, "b"), 

    (2, "c"), (3, "a"), 

    (4, "a"), (5, "c")],

    ["id", "category"])
dddf.show()
#创建StringIndexer对象，设定输入输出参数

indexer =StringIndexer(inputCol ='category', outputCol= 'categoryIndex')

#对这个DataFrame进行训练

model = indexer.fit(dddf)

#利用生成的模型对DataFrame进行transform操作

indexed = model.transform(dddf)

indexed.show()

+---+--------+
| id|category|
+---+--------+
|  0|       a|
|  1|       b|
|  2|       c|
|  3|       a|
|  4|       a|
|  5|       c|
+---+--------+

+---+--------+-------------+
| id|category|categoryIndex|
+---+--------+-------------+
|  0|       a|          0.0|
|  1|       b|          2.0|
|  2|       c|          1.0|
|  3|       a|          0.0|
|  4|       a|          0.0|
|  5|       c|          1.0|
+---+--------+-------------+



In [33]:
# indexed = StringIndexer(inputCol='category', outputCol= "category_index").fit(indexed).transform(indexed)
# indexed.show()
indexed = indexed.drop('category')
rdd1 = indexed.rdd.map(tuple)
rdd1.take(5)

[(0, 0.0, 0.0), (1, 2.0, 2.0), (2, 1.0, 1.0), (3, 0.0, 0.0), (4, 0.0, 0.0)]

In [35]:
def labelData1(data):
    # label: row[end], features: row[0:end-1]
    return data.map(lambda row: LabeledPoint(row[1], row[0:1] + row[2:]))
labelData1(rdd1).take(6)

[LabeledPoint(0.0, [0.0,0.0]),
 LabeledPoint(2.0, [1.0,2.0]),
 LabeledPoint(1.0, [2.0,1.0]),
 LabeledPoint(0.0, [3.0,0.0]),
 LabeledPoint(0.0, [4.0,0.0]),
 LabeledPoint(1.0, [5.0,1.0])]

In [57]:
indexed.select(F.collect_list('categoryIndex')).collect()[0][0]

0.0

In [ ]:
from pyspark.sql import Row
from pyspark.ml.linalg import DenseVector

row = Row("gender", "foo", "bar")

df = sc.parallelize([
  row("0", 3.0, DenseVector([0, 2.1, 'a'])),
  row("1", 1.0, DenseVector([0, 1.1, 'b'])),
  row("1", -1.0, DenseVector([0, 3.4, 'b'])),
  row("0", -3.0, DenseVector([0, 4.1, 'c']))
]).toDF()

In [6]:
schema

['uid',
 'national_identifier',
 'first_name',
 'last_name',
 'mother_first',
 'father_first',
 'gender',
 'birth_city',
 'date_of_birth',
 'id_registration_city',
 'id_registration_district',
 'address_city',
 'address_district',
 'address_neighborhood',
 'street_address',
 'door_or_entrance_number',
 'misc']

In [6]:
columns_to_drop = ['uid',
 'national_identifier',
 'date_of_birth',
 'id_registration_city',
 'id_registration_district',
 'address_city',
 'address_district',
 'address_neighborhood',
 'street_address',
 'door_or_entrance_number',
 'misc']
df = df.drop(*columns_to_drop)

In [7]:
df.show(1)

+----------+---------+------------+------------+------+----------+
|first_name|last_name|mother_first|father_first|gender|birth_city|
+----------+---------+------------+------------+------+----------+
|  NESLIHAN|   ZENGIN|      ZEYCAN|       OSMAN|     K|    KANGAL|
+----------+---------+------------+------------+------+----------+
only showing top 1 row



In [8]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
leftCs=['first_name','last_name','mother_first','father_first','gender','birth_city']
indexer = [StringIndexer(inputCol=column, outputCol=column + "_index") for column in list(set(leftCs))]
indexer

pipeline = Pipeline(stages=indexer)
df = pipeline.fit(df).transform(df)

In [9]:
df.show(1)

+----------+---------+------------+------------+------+----------+------------------+----------------+---------------+------------+------------------+----------------+
|first_name|last_name|mother_first|father_first|gender|birth_city|father_first_index|first_name_index|last_name_index|gender_index|mother_first_index|birth_city_index|
+----------+---------+------------+------------+------+----------+------------------+----------------+---------------+------------+------------------+----------------+
|  NESLIHAN|   ZENGIN|      ZEYCAN|       OSMAN|     K|    KANGAL|               8.0|           164.0|          150.0|         0.0|             450.0|           304.0|
+----------+---------+------------+------------+------+----------+------------------+----------------+---------------+------------+------------------+----------------+
only showing top 1 row



In [10]:
df = df.drop(*leftCs)

In [11]:
df.show(1)

+------------------+----------------+---------------+------------+------------------+----------------+
|father_first_index|first_name_index|last_name_index|gender_index|mother_first_index|birth_city_index|
+------------------+----------------+---------------+------------+------------------+----------------+
|               8.0|           164.0|          150.0|         0.0|             450.0|           304.0|
+------------------+----------------+---------------+------------+------------------+----------------+
only showing top 1 row



In [ ]:
from pyspark.sql import functions as F
len(df.select(F.collect_list('father_first')).collect()[0][0])
# df.select(F.max('father_first_index')).show()
# df.select(F.max('first_name_index')).show()
# df.select(F.max('last_name_index')).show()
# df.select(F.max('mother_first_index')).show()
# df.select(F.max('birth_city_index')).show()

In [ ]:
len(df.select(F.collect_list('first_name')).collect()[0][0])

In [ ]:
len(df.select(F.collect_list('last_name')).collect()[0][0])

In [ ]:
len(df.select(F.collect_list('mother_first')).collect()[0][0])

In [ ]:
len(df.select(F.collect_list('birth_city')).collect()[0][0])

In [ ]:
# +-----------------------+
# |max(father_first_index)|
# +-----------------------+
# |               206561.0|
# +-----------------------+
row = df.select(F.collect_list('father_first_index'),F.collect_list('first_name_index'),F.collect_list('last_name_index'),
                F.collect_list('mother_first_index'),F.collect_list('birth_city_index')).collect()
# print len(df.select(F.collect_list('first_name_index')).collect()[0][0])
# print len(df.select(F.collect_list('last_name_index')).collect()[0][0])
# print len(df.select(F.collect_list('mother_first_index')).collect()[0][0])
# print len(df.select(F.collect_list('birth_city_index')).collect()[0][0])

In [12]:
"""训练模型"""
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import DecisionTree
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
    
def labelData(data):
    # label: row[end], features: row[0:end-1]
    return data.map(lambda row: LabeledPoint(row[3], row[0:3] + row[4:]))

training_data, validation_data, testing_data = labelData(df.rdd).randomSplit([0.7, 0.1 ,0.2])

In [13]:
training_data.take(1)

[LabeledPoint(0.0, [8.0,164.0,150.0,450.0,304.0])]

In [ ]:
model = DecisionTree.trainClassifier(training_data, numClasses=2, maxDepth=2,
                                     categoricalFeaturesInfo={},
                                     impurity='gini', maxBins=100)

print(model.toDebugString())
print('Feature 12:', CV_data.columns[12])
print('Feature 4: ', CV_data.columns[4])

In [35]:
from pyspark.sql.functions import col
# data.groupBy('birth_month').count().orderBy(col('birth_month')).show()

In [19]:
number=birth.select("number").rdd.flatMap(lambda x:x).collect()

In [20]:
month=birth.select("birth_month").rdd.flatMap(lambda x:x).collect()

In [21]:
x=  np.array([c for c in month])
y =  np.array([c for c in number])

In [23]:
b=birth.rdd.map(lambda x:x)
b.take(5)

[Row(birth_month=0, number=57),
 Row(birth_month=1, number=1531),
 Row(birth_month=2, number=952),
 Row(birth_month=3, number=1095),
 Row(birth_month=4, number=780)]

有459个城市？？？

In [ ]:
len(set(list(df['birth_city'])))

In [66]:
d=data.groupBy('birth_city').count().orderBy(col('count').desc())

In [68]:
d1=d.select("birth_city").rdd.flatMap(lambda x:x)

In [72]:
city=[i for i in d1.take(10)]

In [25]:
data.crosstab('first_name', 'gender').show()

+-----------------+---+---+
|first_name_gender|  E|  K|
+-----------------+---+---+
|       ELIF GAMZE|  0|  1|
|         BELGUZAR|  0|  1|
|  KADIR HAYRETTIN|  1|  0|
|        MEVLUDUYE|  0|  1|
|           EBUZER|  1|  0|
|             UGUR| 13|  1|
|            SENEM|  0| 18|
|             EMEL|  0|  5|
|            IDRIS|  2|  0|
|             ZEKI|  7|  0|
|         GULENBER|  0|  1|
|           SEYCAN|  0|  1|
|     HASAN TAHSIN|  1|  0|
|           NURSEN|  0|  8|
|            KAFUR|  1|  0|
|         ALPASLAN|  2|  0|
|         NURETTIN| 17|  0|
|        ABDULGANI|  1|  0|
|            MUSAB|  1|  0|
|          GULUZAR|  0|  6|
+-----------------+---+---+
only showing top 20 rows

